In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing 
from sklearn import compose

In [ ]:
df = pd.read_csv("/home/jordan/Downloads/Titanic/train.csv")

In [ ]:
X = df.copy()
X = X.dropna()
Y = X[['Survived', 'PassengerId']].copy()


In [ ]:
Y = Y.merge(X, 'inner', on = 'PassengerId')
Y = Y.pop('Survived_y')
X = X.drop('Survived', axis=1)
X

In [ ]:
train = X.iloc[:163]
test = X.iloc[163:]

train_label = Y.iloc[:163]
test_label = Y.iloc[163:]

In [ ]:
train.drop('Name', inplace=True, axis=1)

In [ ]:
ohe = preprocessing.OneHotEncoder()
cols = ['Sex', 'Ticket', 'Cabin', 'Embarked']
for x, i in enumerate(cols):
    train[[i]] = ohe.fit_transform(train[[i]])




In [ ]:
transformer = compose.make_column_transformer(
    (preprocessing.Normalizer(),
    ['Pclass','Age', 'SibSp', 'Parch', 'Fare']),
    remainder = 'passthrough'
)

In [ ]:
transformed = transformer.fit_transform(train)
transformed_df = pd.DataFrame(transformed)

In [ ]:
transformed_df = np.asarray(transformed_df).astype(np.float32)

In [ ]:
def get_model():
    model = tf.keras.Sequential([
            tf.keras.layers.Dense(20, activation='tanh'),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer='adam',
            loss='BinaryCrossentropy',
            metrics='binary_accuracy')
    return model

In [ ]:
model = get_model()       
history = model.fit(transformed_df, train_label, epochs=5, batch_size=128)